# Create a Twisted Interface between Nanoribbons

Use nanoribbon interface builder to create twisted interfaces between two nanoribbons.

<h2 style="color:green">Usage</h2>

1. Make sure to select Input Materials (in the outer runtime) before running the notebook.
1. Set notebook parameters in cell 1.1. below (or use the default values).
1. Click "Run" > "Run All" to run all cells. 
1. Wait for the run to complete (depending on the parameters can take a few min). 
1. Scroll down to view results. 

## Summary
1. Prepare the Environment: Set up the notebook and install packages, preview the input materials
1. Create and visualize the initial nanoribbons
1. Generate twisted interface between nanoribbons
1. Visualize the final interface

## Notes
1. Nanoribbons are created from the input materials with specified width and length.
1. The top nanoribbon is rotated by the specified twist angle.
1. Vacuum is added along x and y directions to isolate the ribbons.
1. For more information, see [Introduction](Introduction.ipynb)


## 1. Prepare the Environment
### 1.1. Set up the notebook 
Set the following flags to control the notebook behavior 


In [ ]:
# Material selection and basic parameters
FILM_INDEX = 0  # Index in the list of materials, to access as materials[FILM_INDEX]
SUBSTRATE_INDEX = None  # Can be None to use same material as film

# Interface parameters
TWIST_ANGLE = 15.0  # in degrees
INTERFACE_DISTANCE = 3.0  # in Angstroms
INTERFACE_VACUUM = 20.0  # in Angstroms

# Nanoribbon parameters
RIBBON_WIDTH = 5  # Width of the nanoribbon in unit cells
RIBBON_LENGTH = 10  # Length of the nanoribbon in unit cells
VACUUM_X = 5.0  # Vacuum along x on both sides, in Angstroms
VACUUM_Y = 5.0  # Vacuum along y on both sides, in Angstroms

# Visualization parameters
SHOW_INTERMEDIATE_STEPS = True
VISUALIZE_REPETITIONS = [1, 1, 1]

### 1.2. Install packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` (see [README](../../README.ipynb)).

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip
  
    await micropip.install('mat3ra-api-examples', deps=False)
    from utils.jupyterlite import install_packages
    await install_packages("")

### 1.3. Load and preview input materials


In [ ]:
from utils.jupyterlite import get_materials

materials = get_materials(globals())

## 2. Prepare Materials
### 2.1. Select and visualize initial materials


In [ ]:
from utils.visualize import visualize_materials

film = materials[FILM_INDEX]
substrate = materials[SUBSTRATE_INDEX] if SUBSTRATE_INDEX is not None else film

if SHOW_INTERMEDIATE_STEPS:
    print("\nInitial material structure:")
    visualize_materials(film, repetitions=VISUALIZE_REPETITIONS)

## 3. Generate Nanoribbon Interface
### 3.1. Set up interface configuration and builder


In [ ]:
from mat3ra.made.tools.build.interface.builders import NanoRibbonTwistedInterfaceBuilder
from mat3ra.made.tools.build.interface.configuration import NanoRibbonTwistedInterfaceConfiguration

config = NanoRibbonTwistedInterfaceConfiguration(
    film=film,
    substrate=substrate,
    twist_angle=TWIST_ANGLE,
    distance_z=INTERFACE_DISTANCE,
    ribbon_width=RIBBON_WIDTH,
    ribbon_length=RIBBON_LENGTH,
    vacuum_x=VACUUM_X,
    vacuum_y=VACUUM_Y
)

builder = NanoRibbonTwistedInterfaceBuilder()

### 3.2. Generate and analyze interface


In [ ]:
interface = builder.get_material(config)

print("\nInterface created successfully!")
print(f"Twist angle: {TWIST_ANGLE}°")
print(f"Number of atoms: {len(interface.basis.elements.ids)}")
print(f"Cell vectors:\n{interface.basis.cell.vectors_as_array}")

if SHOW_INTERMEDIATE_STEPS:
    visualize_materials(interface, repetitions=VISUALIZE_REPETITIONS)
    visualize_materials(interface, repetitions=VISUALIZE_REPETITIONS, rotation="-90x")

## 4. Pass data to the outside runtime


In [ ]:
from utils.jupyterlite import set_materials

set_materials(interface)